<a href="https://colab.research.google.com/github/gillesvtsilvano/GANs/blob/master/GANS_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse exemplo segue o tutorial que pode ser encontrado [aqui](https://hub.packtpub.com/generative-adversarial-networks-using-keras/).

Para trabalhos futuros, observar:
* [Step-by-step](https://www.kdnuggets.com/2017/10/seven-steps-deep-learning-keras.html).
* Repositório do [Keras implementations of Generative Adversarial Networks](https://github.com/eriklindernoren/Keras-GAN)

# Introdução

O dataset utilizado será o MNIST (Modified National Institute of Standards and Technology database), dataset clássico contendo 60,000 caracteres números (0-9) escritos à mão. Esse dataset não oferece nenhum desafio (já foram superados), porém, é um ótimo exemplo para prova de conceito ou para aprender alguma conteúdo relacionado ao aprendizado de máquina, nesse caso,  Generative Adversarial Networks (GANs).

O framework [Keras](https://keras.io/) já se encarregou de organizar o dataset, dividindo-o em duas partes: uma para treinamento, contendo 50,000 imagens, e outra para testes, contendo 10,000 imagens. 

In [1]:
from keras.datasets import mnist #Loading MNIST dataset

Using TensorFlow backend.


<span style="color: red">TODO</span>
As GANs são compostas por duas redes neurais, uma Geradora (Generative) e outra Descritiva, que jogam _minmax_ uma contra a outra (Adversarial), no intuito de que a Descritiva evolua com base nos exemplos gerados pela Geradora, que aprenderá cada vez mais a gerar melhores exemplos, melhorando a qualidade do aprendizado da Descritiva.



In [0]:
def load_data():
  (X_train, _), (_, _) = mnist.load_data()
  """Um pixel vai de 0~255, mas a função de ativação do nosso exemplo
  irá de -1 até 1¹"""
  X_train = (X_tran.astype(np.float32) - 127.5) / 127.5
  """Não entendi. Alguma coisa com não utilzar os labels.
  Resposta no Livro Deep Learning Quick Reference²"""
  X_train = np.expand_dims(X_train, axis=3)
  return X_train

¹[Artigo sobre Standardization](http://sebastianraschka.com/Articles/2014_about_feature_scaling.html)<br>
²[Deep Learning Quick Reference](https://www.amazon.com/Deep-Learning-Quick-Reference-optimizing-ebook/dp/B0791JRGPY)

# Constuindo a Rede Geradora

Primeiro o código:

In [0]:
def build_generator(noise_shape=(100, )):
  inpt = Input(noise_shape)
  x = Dense(128 * 7 * 7, activation="relu")(inpt)
  x = Reshape((7, 7, 128))(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = UpSampling2D()(x)
  x = Conv2D(128, kernel_size=3, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = UpSampling2D()(x)
  x = Conv2D(64, kernel_size=3, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = Conv2D(1, kernel_size=3, padding="same")(x)
  out = Activation("tanh")(x)
  model = Model(inpt, out)
  print("--- Generator ---")
  model.summary()
  return model

We have not previously used the UpSampling2D layer. This layer will take increases in the rows and columns of the input tensor, leaving the channels unchanged. It does this by repeating the values in the input tensor. By default, it will double the input. If we give an UpSampling2D layer a 7 x 7 x 128 input, it will give us a 14 x 14 x 128 output.

Typically when we build a CNN, we start with an image that is very tall and wide and uses convolutional layers to get a tensor that’s very deep but less tall and wide. Here we will do the opposite. We’ll use a dense layer and a reshape to start with a 7 x 7 x 128 tensor and then, after doubling it twice, we’ll be left with a 28 x 28 tensor. Since we need a grayscale image, we can use a convolutional layer with a single unit to get a 28 x 28 x 1 output.

This sort of generator arithmetic is a little off-putting and can seem awkward at first but after a few painful hours, you will get the hang of it!

# Construindo a Descriminante

A Descriminante é, em grande parte, a mesma que qualquer outra CNN. 

In [0]:
def build_descriminator(img_shape):
  inpt = Input(img_shape)
  x = Conv2D(32, kernel_size=3, strides=2, padding="same")(inpt)
  x = LeakyReLU(alpha=0.2)(x)
  x = Dropout(0.25)(x)
  x = Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
  x = ZeroPadding2D(padding=((0, 1), (0, 1)))(x)
  x = (LeakyReLU(alpha=0.2))(x)
  x = Dropout(0.25)(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = Conv2D(128, kernel_size=3, strides=2, padding="same")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = Conv2D(256, kernel_size=3, strides=1, padding="same")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = Dropout(0.25)(x)
  x = Flatten()(x)
  out = Dense(1, activation='sigmoid')(x)
  model = Model(inpt, out)
  print("--- DIscriminator ---")
  model.summary()
  return model